<a href="https://colab.research.google.com/github/hoangthanh300405-ops/football-ai-colab-/blob/main/ranking_players.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
import numpy as np

In [19]:
df = pd.read_csv('lineups.csv')
df.head(15)

,eventId,teamId,homeAway,starter,athleteDisplayName,Pos,Age,Min/Match,Save%,GA/Match,...,G+A/Match,xAG,PrgC,PrgP,KP/Match,G+A-PK/Match,npxG,PrgR,SoT%,Sh/90
0,704279,Manchester United,home,1,André Onana,GK,28.0,90.00,68.9,1.29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,704279,Manchester United,home,1,Noussair Mazraoui,DF,26.0,76.70,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,704279,Manchester United,home,1,Diogo Dalot,DF,25.0,85.27,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,704279,Manchester United,home,1,Kobbie Mainoo,MF,19.0,66.04,NaN,NaN,...,0.00,1.6,22.0,65.0,0.56,NaN,NaN,NaN,NaN,NaN
4,704279,Manchester United,home,1,Harry Maguire,DF,31.0,65.11,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,704279,Manchester United,home,1,Lisandro Martínez,DF,26.0,87.55,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,704279,Manchester United,home,1,Amad,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,704279,Manchester United,home,1,Casemiro,MF,32.0,62.25,NaN,NaN,...,0.04,2.3,4.0,84.0,0.92,NaN,NaN,NaN,NaN,NaN
8,704279,Manchester United,home,1,Bruno Fernandes,MF,29.0,83.83,NaN,NaN,...,0.50,8.5,77.0,325.0,2.53,NaN,NaN,NaN,NaN,NaN
9,704279,Manchester United,home,1,Mason Mount,MF,25.0,36.59,NaN,NaN,...,0.06,0.6,15.0,33.0,0.47,NaN,NaN,NaN,NaN,NaN


In [20]:
import pandas as pd


def normalize(col):
    valid = col.dropna()
    if valid.empty or valid.max() == valid.min():
        return pd.Series(0.0, index=col.index)
    return ((col - valid.min()) / (valid.max() - valid.min())).fillna(0)

df['Pos'] = df['Pos'].fillna('').astype(str)

#Persional Point

df['GA_n']    = normalize(df['G+A/Match'])
df['xAG_n']   = normalize(df['xAG'])
df['PrgC_n']  = normalize(df['PrgC'])

df['ATK'] = 100 * (
    0.50 * df['GA_n'] +
    0.30 * df['xAG_n'] +
    0.20 * df['PrgC_n']
)


df['Clr_n'] = normalize(df['Clr/Match'])
df['Tkl_n'] = normalize(df['TklW%'])
df['Blk_n'] = normalize(df['Blocks/Match'])
df['Int_n'] = normalize(df['Int/Match'])
df['Err_n'] = normalize(df['Err/Match'])

df['DEF'] = 100 * (
    0.30 * df['Tkl_n'] +
    0.25 * df['Int_n'] +
    0.20 * df['Clr_n'] +
    0.15 * df['Blk_n'] -
    0.10 * df['Err_n']
)


df['Save_n'] = normalize(df['Save%'])
df['CS_n']   = normalize(df['CS%'])
df['GA_gk_n'] = normalize(df['GA/Match'])
df['PK_n']   = normalize(df['PKsv%'])

df['GK'] = 100 * (
    0.35 * df['Save_n'] +
    0.25 * df['CS_n'] +
    0.20 * df['PK_n'] -
    0.20 * df['GA_gk_n']
)


#Rating
def calc_rating(row):
    pos = row['Pos']

    if pos == '':
        return 0
    if 'GK' in pos:
        return row['GK']
    if 'DF' in pos and 'MF' in pos:
        return 0.65 * row['DEF'] + 0.35 * row['ATK']
    if 'MF' in pos and 'FW' in pos:
        return 0.70 * row['ATK'] + 0.30 * row['DEF']
    if 'DF' in pos:
        return row['DEF']
    if 'MF' in pos:
        return 0.60 * row['ATK'] + 0.40 * row['DEF']
    if 'FW' in pos:
        return row['ATK']
    return 0

df['Rating'] = df.apply(calc_rating, axis=1)



df.loc[df['Min/Match'] < 30, ['ATK','DEF','GK','Rating']] *= 0.5

#TEAM

atk_team = (
    df[df['Pos'].str.contains('FW|MF')]
    .groupby('teamId')['ATK']
    .mean()
    .rename('ATK_team')
)

def_team = (
    df[df['Pos'].str.contains('DF|MF')]
    .groupby('teamId')['DEF']
    .mean()
    .rename('DEF_team')
)

gk_team = (
    df[df['Pos'].str.contains('GK')]
    .groupby('teamId')['GK']
    .mean()
    .rename('GK_team')
)

df = df.merge(atk_team, on='teamId', how='left')
df = df.merge(def_team, on='teamId', how='left')
df = df.merge(gk_team, on='teamId', how='left')

df.head(20)


,eventId,teamId,homeAway,starter,athleteDisplayName,Pos,Age,Min/Match,Save%,GA/Match,...,DEF,Save_n,CS_n,GA_gk_n,PK_n,GK,Rating,ATK_team,DEF_team,GK_team
0,704279,Manchester United,home,1,André Onana,GK,28.0,90.00,68.9,1.29,...,0.000000,0.481667,0.265,0.43,0.25,19.883333,19.883333,21.216109,19.559019,19.883333
1,704279,Manchester United,home,1,Noussair Mazraoui,DF,26.0,76.70,NaN,NaN,...,41.314408,0.000000,0.000,0.00,0.00,0.000000,41.314408,21.216109,19.559019,19.883333
2,704279,Manchester United,home,1,Diogo Dalot,DF,25.0,85.27,NaN,NaN,...,41.643593,0.000000,0.000,0.00,0.00,0.000000,41.643593,21.216109,19.559019,19.883333
3,704279,Manchester United,home,1,Kobbie Mainoo,MF,19.0,66.04,NaN,NaN,...,0.000000,0.000000,0.000,0.00,0.00,0.000000,4.450421,21.216109,19.559019,19.883333
4,704279,Manchester United,home,1,Harry Maguire,DF,31.0,65.11,NaN,NaN,...,43.860164,0.000000,0.000,0.00,0.00,0.000000,43.860164,21.216109,19.559019,19.883333
5,704279,Manchester United,home,1,Lisandro Martínez,DF,26.0,87.55,NaN,NaN,...,51.113031,0.000000,0.000,0.00,0.00,0.000000,51.113031,21.216109,19.559019,19.883333
6,704279,Manchester United,home,1,Amad,,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000,0.00,0.00,0.000000,0.000000,21.216109,19.559019,19.883333
7,704279,Manchester United,home,1,Casemiro,MF,32.0,62.25,NaN,NaN,...,0.000000,0.000000,0.000,0.00,0.00,0.000000,6.001932,21.216109,19.559019,19.883333
8,704279,Manchester United,home,1,Bruno Fernandes,MF,29.0,83.83,NaN,NaN,...,0.000000,0.000000,0.000,0.00,0.00,0.000000,43.802964,21.216109,19.559019,19.883333
9,704279,Manchester United,home,1,Mason Mount,MF,25.0,36.59,NaN,NaN,...,0.000000,0.000000,0.000,0.00,0.00,0.000000,5.036202,21.216109,19.559019,19.883333


In [21]:
df.to_csv("lineups.csv", index=False)
